In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        break

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/final-masks/Pred_Images_for_TN_1463-20250328T162926Z-001/Pred_Images_for_TN_1463/Unet_1463/__huggingface_repos__.json
/kaggle/input/final-masks/Pred_Images_for_TN_1463-20250328T162926Z-001/Pred_Images_for_TN_1463/Unet_1463/Unet/predicted_masks/pred_20191129T103133_749_mask.png
/kaggle/input/final-masks/Pred_Images_for_TN_1463-20250328T162926Z-001/Pred_Images_for_TN_1463/Unet_1463/__results___files/__results___5_0.png
/kaggle/input/final-masks/Pred_Images_for_TN_1463-20250328T162926Z-001/Pred_Images_for_TN_1463/TransUnet_1463/__huggingface_repos__.json
/kaggle/input/final-masks/Pred_Images_for_TN_1463-20250328T162926Z-001/Pred_Images_for_TN_1463/TransUnet_1463/TransUnet/predicted_masks/pred_20191129T103133_749_mask.png
/kaggle/input/final-masks/Pred_Images_for_TN_1463-20250328T162926Z-001/Pred_Images_for_TN_1463/TransUnet_1463/__results___files/__results___5_0.png
/kaggle/input/final-masks/Pred_Images_for_TN_1463-20250328T162926Z-001/Pred_Images_for_TN_1463/TransFuse_1463/

In [2]:
import numpy as np
import cv2


def find_tangent_lines(ellipse, reference_point):
    """Find the Ellipse's two tangents that go through a reference point

    Args:
        ellipse:
          center: The center of the ellipse
          axes: Major and Minor axes of the ellipse
          rotation: The counter-clockwise rotation of the ellipse in radians
        reference_point: The coordinates of the reference point.

    Return:
        (m1, h1): Slope and intercept of the first tangent.
        (m2, h2): Slope and intercept of the second tangent.
    """
    (x0, y0), axes, rotation  = ellipse
    a, b = axes[0]/2, axes[1]/2
    rotation =  np.radians(rotation)
    s, c = np.sin(rotation), np.cos(rotation)
    p0, q0 = reference_point

    A = (-a ** 2 * s ** 2 - b ** 2 * c ** 2 + (y0 - q0) ** 2)
    B = 2 * (c * s * (a ** 2 - b ** 2) - (x0 - p0) * (y0 - q0))
    C = (-a ** 2 * c ** 2 - b ** 2 * s ** 2 + (x0 - p0) ** 2)

    if B ** 2 - 4 * A * C < 0:
        raise ValueError('Reference point lies inside the ellipse')

    t1, t2 = (
        (-B + np.sqrt(B ** 2 - 4 * A * C)) / (2 * A),
        (-B - np.sqrt(B ** 2 - 4 * A * C)) / (2 * A),
    )
    return (
        (1 / t1, q0 - p0 / t1),
        (1 / t2, q0 - p0 / t2),
    )


# Coordinates of the ellipse's major axes
def get_major_axis(ellipse):

    center, axes, angle = ellipse
    major_axis_angle_rad = np.radians(angle)
    major_axis_length = max(axes)
    cos_angle = np.cos(major_axis_angle_rad)
    sin_angle = np.sin(major_axis_angle_rad)

    major_axis_endpoint1 = (
        int(center[0] - 0.5 * major_axis_length * sin_angle),
        int(center[1] + 0.5 * major_axis_length * cos_angle)
    )
    major_axis_endpoint2 = (
        int(center[0] + 0.5 * major_axis_length * sin_angle),
        int(center[1] - 0.5 * major_axis_length * cos_angle)
    )

    return major_axis_endpoint1, major_axis_endpoint2

# Angle between 2 lines
def calculate_angle(lineA, lineB):

    vA = np.array([(lineA[0][0] - lineA[1][0]), (lineA[0][1] - lineA[1][1])])
    vB = np.array([(lineB[0][0] - lineB[1][0]), (lineB[0][1] - lineB[1][1])])

    angle = np.arccos(np.dot(vA, vB) / np.dot(vA, vA) ** 0.5 / np.dot(vB, vB) ** 0.5)
    deg = np.rad2deg(angle) 

    if deg - 180 >= 0:
        return 360 - deg
    else:
        return deg
        
        
def angle_of_progression_estimation(label, return_img=False):
    tmp_img = np.zeros_like(label).astype(np.uint8)
    
    # PS ellipse
    contours_ps, _ = cv2.findContours((label.cpu().numpy() == 1).astype(np.uint8), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    if len(contours_ps) == 0:
        print("Skipping PS contour due to no contours found.")
        return (None, None) if return_img else None

    largest_contour_ps = max(contours_ps, key=cv2.contourArea)

    if len(largest_contour_ps) < 5:
        print("Skipping PS contour due to insufficient points.")
        return (None, None) if return_img else None

    ellipse_ps = cv2.fitEllipse(largest_contour_ps)
    
    # PS ellipse: major axis points
    (x1, y1), (x2, y2) = get_major_axis(ellipse_ps)
    ps_points = [(x1, y1), (x2, y2)] if x1 > x2 else [(x2, y2), (x1, y1)]
    ps_pont = (int(ps_points[0][0]), int(ps_points[0][1]))
    
    # FH ellipse
    contours_fh, _ = cv2.findContours((label.cpu().numpy() == 2).astype(np.uint8), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    if len(contours_fh) == 0:
        print("Skipping FH contour due to no contours found.")
        return (None, None) if return_img else None

    largest_contour_fh = max(contours_fh, key=cv2.contourArea)

    if len(largest_contour_fh) < 5:
        print("Skipping FH contour due to insufficient points.")
        return (None, None) if return_img else None

    ellipse_fh = cv2.fitEllipse(largest_contour_fh)

    # Find tangent line
    try:
        (m1, h1), (m2, h2) = find_tangent_lines(ellipse=ellipse_fh, reference_point=ps_pont)
    except ValueError:
        print("Skipping due to invalid tangent calculation.")
        return (None, None) if return_img else None

    tmp_p_disp = 150
    op = ((ps_points[0][1] + tmp_p_disp - h1) / m1)
    
    # Draw if needed
    if return_img:
        cv2.ellipse(tmp_img, ellipse_ps, (120, 255, 255), 1, cv2.LINE_AA) 
        cv2.line(tmp_img, (int(x1), int(y1)), (int(x2), int(y2)), (160, 0, 255), 2) 
        cv2.ellipse(tmp_img, ellipse_fh, (160, 255, 255), 1, cv2.LINE_AA)
        cv2.line(tmp_img, ps_pont, (int(op), int(ps_points[0][1] + tmp_p_disp)), (240, 0, 0), 2)

    # Angle estimation
    aop = calculate_angle([ps_points[0], ps_points[1]], [ps_pont, (op, ps_points[0][1] + tmp_p_disp)])
    
    return (aop, tmp_img) if return_img else aop


In [3]:
import os
import random

# Paths
gt_path = "/kaggle/input/ground-truth-mask-img/testMaskResnet18Cbam-20250329T151022Z-001/testMaskResnet18Cbam"
pred_path = "/kaggle/input/final-masks/Pred_Images_for_TN_1470-20250328T201100Z-001/Pred_Images_for_TN_1470/Attention_unet_1470/AttentionUnet/predicted_masks"

# Get filenames
gt_files = sorted(os.listdir(gt_path))
pred_files = sorted(os.listdir(pred_path))

# Remove "pred_" from predicted filenames
clean_pred_files = [fname.replace("pred_", "") for fname in pred_files]

# Find common filenames
common_images = list(set(gt_files) & set(clean_pred_files))

print(f"Total matching masks: {len(common_images)}")  # Check how many match

# Select random matching images
num_images = min(5, len(common_images))  # Load up to 5
random_images = random.sample(common_images, num_images)

# Create matched pairs
matched_pairs = [(img, f"pred_{img}") for img in random_images]

# Print matched pairs for verification
for gt_file, pred_file in matched_pairs:
    print(f"GT: {gt_file}  -->  Pred: {pred_file}")


Total matching masks: 1470
GT: 04218PSFHSAoP1358_475_mask.png  -->  Pred: pred_04218PSFHSAoP1358_475_mask.png
GT: 20191129T110732_739_mask.png  -->  Pred: pred_20191129T110732_739_mask.png
GT: 01943PSFHSAoP4k_1943_mask.png  -->  Pred: pred_01943PSFHSAoP4k_1943_mask.png
GT: 20200103T104919_649_mask.png  -->  Pred: pred_20200103T104919_649_mask.png
GT: 20191220T112127_169_mask.png  -->  Pred: pred_20191220T112127_169_mask.png


In [4]:
import os

# Set output paths
save_dir_working = '/kaggle/working/aop_visuals'
save_dir_output = '/kaggle/outputs/aop_visuals'

# Ensure directories exist
os.makedirs(save_dir_working, exist_ok=True)
os.makedirs(save_dir_output, exist_ok=True)

# Function to save image to both locations
def save_aop_images(gt_name_base, gt_img, pred_img):
    # Create base names
    base_name = os.path.splitext(gt_name_base)[0]  # Remove .png
    
    # Construct full paths
    gt_path_working = os.path.join(save_dir_working, f'{base_name}_gtAoP.png')
    pred_path_working = os.path.join(save_dir_working, f'{base_name}_predAoP.png')

    gt_path_output = os.path.join(save_dir_output, f'{base_name}_gtAoP.png')
    pred_path_output = os.path.join(save_dir_output, f'{base_name}_predAoP.png')

    # Save both images (GT and Pred)
    cv2.imwrite(gt_path_working, gt_img)
    cv2.imwrite(pred_path_working, pred_img)
    cv2.imwrite(gt_path_output, gt_img)
    cv2.imwrite(pred_path_output, pred_img)


In [5]:
# import os
# import random
# import numpy as np
# import cv2  # Import OpenCV
# import matplotlib.pyplot as plt
# import torch

# # Create matched pairs
# matched_pairs = [(img, f"pred_{img}") for img in common_images]

# meanAoP = []
# skipped_images_count = 0
# # Load and send images to function
# for idx, (gt_name, pred_name) in enumerate(matched_pairs):
#     # print(pred_name)

#     # Read images with OpenCV
#     gt_img = cv2.imread(os.path.join(gt_path, gt_name), 0)  # Load as grayscale
#     pred_img = cv2.imread(os.path.join(pred_path, pred_name), 0)  # Load as grayscale
    
#     # Check if images are loaded properly
#     if gt_img is None or pred_img is None:
#         print(f"Error loading images: {gt_name}, {pred_name}")
#         continue
    
#     gt_img = cv2.resize(gt_img, (224, 224), interpolation=cv2.INTER_NEAREST)  # Use INTER_NEAREST for categorical images
#     pred_img = cv2.resize(pred_img, (224, 224), interpolation=cv2.INTER_NEAREST)  # Use INTER_NEAREST for categorical images

#     # Convert to torch tensors
#     pred_img = torch.from_numpy(pred_img).long()
#     gt_img = torch.from_numpy(gt_img).long()

#     # Print unique values in gt_img and pred_img
#     # print(torch.unique(pred_img))
    
#     # Skip if the image contains only two unique classes
#     if len(torch.unique(gt_img)) == 2 or len(torch.unique(pred_img)) == 2:
#         skipped_images_count += 1
#         continue

#     # Define mapping: 0 → 0, 76 → 1, 127 → 1, 149 → 2, 255 → 2
#     mapping_gt = {0: 0, 76: 1, 127: 1, 149: 2, 255: 2}
#     mapping_pred = {0: 0, 127: 1, 255: 2}

#     # Apply mapping using np.vectorize
#     pred_img = np.vectorize(mapping_pred.get)(pred_img).astype(np.uint8)
#     gt_img = np.vectorize(mapping_gt.get)(gt_img).astype(np.uint8)

#     # Convert to torch tensors
#     pred_img = torch.from_numpy(pred_img).long()
#     gt_img = torch.from_numpy(gt_img).long()

#     # Angle of Progression Estimation
#     gtAoP = angle_of_progression_estimation(gt_img, return_img=True)
#     predAoP = angle_of_progression_estimation(pred_img, return_img=True)

#     # Check if either AoP value is None, and skip if true
#     if gtAoP is None or predAoP is None:
#         # print(f"Skipping image pair {gt_name} and {pred_name} due to invalid AoP calculation.")
#         skipped_images_count += 1
#         continue
#     gt_result = angle_of_progression_estimation(gt_img, return_img=True)
#     pred_result = angle_of_progression_estimation(pred_img, return_img=True)
    
#     if gt_result is None or pred_result is None:
#         # print(f"Skipping image pair {gt_name} and {pred_name} due to invalid AoP.")
#         skipped_images_count += 1
#         continue
    
#     gtAoP, gtAoPimg = gt_result
#     predAoP, predAoPimg = pred_result
    
#     # Ensure AoP images are valid (non-empty)
#     if gtAoPimg is None or predAoPimg is None or gtAoPimg.size == 0 or predAoPimg.size == 0:
#         # print(f"Skipping save due to empty AoP image for {gt_name}.")
#         skipped_images_count += 1
#         continue
    
#     # Save AoP annotated images
#     # save_aop_images(gt_name, gtAoPimg, predAoPimg)
    
#     # Append AoP difference
#     meanAoP.append(abs(gtAoP - predAoP))

# # Calculate and print the mean AoP
# if meanAoP:
#     print(f"Mean AoP: {np.mean(meanAoP)}")
# else:
#     print("No valid AoP calculations were made.")

# # Print skipped images count
# print(f"Skipped images count: {skipped_images_count}")


In [6]:
# print(len(meanAoP))

In [7]:
# import os
# import random
# import numpy as np
# import cv2  # Import OpenCV
# import matplotlib.pyplot as plt
# import torch

# # Select random images

# # Select random matching images
# num_images = min(5, len(common_images))  # Load up to 5
# random_images = random.sample(common_images, num_images)

# # Create matched pairs
# matched_pairs = [(img, f"pred_{img}") for img in random_images]

# random_images = random.sample(matched_pairs, num_images)

# # Create a figure with 5 rows and 4 columns
# fig, axs = plt.subplots(nrows=num_images, ncols=4, figsize=(20, num_images * 4))
# skipped_images_count = 0
# # Load and send images to function
# for idx, (gt_name, pred_name) in enumerate(matched_pairs):
#     print(gt_name)
#     print(pred_name)

#     # Read images with OpenCV
#     gt_img = cv2.imread(os.path.join(gt_path, gt_name), 0)  # Load as grayscale
#     pred_img = cv2.imread(os.path.join(pred_path, pred_name), 0)  # Load as grayscale
    
#     # Check if images are loaded properly
#     # if gt_img is None or pred_img is None:
#     #     print(f"Error loading images: {gt_name}, {pred_name}")
#     #     continue
#      # Convert to torch tensors
#     # pred_img = torch.from_numpy(pred_img).long()
#     # gt_img = torch.from_numpy(gt_img).long()

#     # # Print unique values in gt_img and pred_img
#     # print(torch.unique(gt_img))
#     # print(torch.unique(pred_img))
#     # Resize both images to 224x224
#     gt_img = cv2.resize(gt_img, (224, 224), interpolation=cv2.INTER_NEAREST)  # Use INTER_NEAREST for categorical images
#     pred_img = cv2.resize(pred_img, (224, 224), interpolation=cv2.INTER_NEAREST)  # Use INTER_NEAREST for categorical images

#      # Convert to torch tensors
#     pred_img = torch.from_numpy(pred_img).long()
#     gt_img = torch.from_numpy(gt_img).long()

#     # Print unique values in gt_img and pred_img
#     print(torch.unique(gt_img))
#     print(torch.unique(pred_img))
#     # Skip if the image contains only two unique classes
#     if len(torch.unique(gt_img)) == 2 or len(torch.unique(pred_img)) == 2:
#         skipped_images_count += 1
#         print(f"Skipping image {gt_name} and {pred_name} as they contain only 2 unique classes.")
#         continue
#     # Define mapping: 0 → 0, 76 → 1, 127 → 1, 149 → 2, 255 → 2
#     mapping_gt = {0: 0, 76: 1, 127: 1, 149: 2, 255: 2}
#     mapping_pred = {0: 0, 127: 1, 255: 2}

#     # Apply mapping using np.vectorize
#     pred_img = np.vectorize(mapping_pred.get)(pred_img).astype(np.uint8)
#     gt_img = np.vectorize(mapping_gt.get)(gt_img).astype(np.uint8)

#     # Convert to torch tensors
#     pred_img = torch.from_numpy(pred_img).long()
#     gt_img = torch.from_numpy(gt_img).long()

#     # Print unique values in gt_img and pred_img
#     print(torch.unique(gt_img))
#     print(torch.unique(pred_img))

#     # Ground Truth AoP (Column 0)
#     axs[idx, 0].imshow(gt_img.numpy(), cmap='gray')
#     axs[idx, 0].set_title(f"GT img: {gt_name}")
#     axs[idx, 0].axis('off')

#     # Predicted Mask AoP (Column 1)
#     gtAoP, gtAoPimg = angle_of_progression_estimation(gt_img, return_img=True)
#     axs[idx, 1].imshow(gtAoPimg, cmap='gray')
#     axs[idx, 1].set_title(f"GT AoP: {round(gtAoP, 2)}°")
#     axs[idx, 1].axis('off')

#     # Ground Truth Mask (Column 2)
#     axs[idx, 2].imshow(pred_img.numpy(), cmap='gray')
#     axs[idx, 2].set_title(f"Pred img: {pred_name}")
#     axs[idx, 2].axis('off')

#     # Predicted Mask AoP (Column 3)
#     predAoP, predAoPimg = angle_of_progression_estimation(pred_img, return_img=True)
#     axs[idx, 3].imshow(predAoPimg, cmap='gray')
#     axs[idx, 3].set_title(f"Pred AoP: {round(predAoP, 2)}°")
#     axs[idx, 3].axis('off')

# plt.tight_layout()
# plt.show()
# print(skipped_images_count)

## metrics

## unet

In [8]:
import os
import random

# Paths
gt_path = "/kaggle/input/ground-truth-mask-img/test_Masks-20250329T151233Z-001/test_Masks"
pred_path = "/kaggle/input/final-masks/Pred_Images_for_TN_1414-20250328T162906Z-001/Pred_Images_for_TN_1414/unet_1414/Unet/predicted_masks"

# Get filenames
gt_files = sorted(os.listdir(gt_path))
pred_files = sorted(os.listdir(pred_path))

# Remove "pred_" from predicted filenames
clean_pred_files = [fname.replace("pred_", "") for fname in pred_files]

# Find common filenames
common_images = list(set(gt_files) & set(clean_pred_files))

print(f"Total matching masks: {len(common_images)}")  # Check how many match

# Select random matching images
num_images = min(5, len(common_images))  # Load up to 5
random_images = random.sample(common_images, num_images)

# Create matched pairs
matched_pairs = [(img, f"pred_{img}") for img in random_images]

# Print matched pairs for verification
for gt_file, pred_file in matched_pairs:
    print(f"GT: {gt_file}  -->  Pred: {pred_file}")


Total matching masks: 1414
GT: 00793_PSFHAoP4k_mask.png  -->  Pred: pred_00793_PSFHAoP4k_mask.png
GT: 04293_PSFHAoP1358_mask.png  -->  Pred: pred_04293_PSFHAoP1358_mask.png
GT: 03997_PSFHAoP4k_mask.png  -->  Pred: pred_03997_PSFHAoP4k_mask.png
GT: 20190918T120011_49_mask.png  -->  Pred: pred_20190918T120011_49_mask.png
GT: 20191214T103803_479_mask.png  -->  Pred: pred_20191214T103803_479_mask.png


In [9]:
import os
import random
import numpy as np
import cv2  # Import OpenCV
import matplotlib.pyplot as plt
import torch

# Create matched pairs
matched_pairs = [(img, f"pred_{img}") for img in common_images]

meanAoP = []  # For storing the absolute differences
mean_pred_aop = []  # For storing predicted AoP values
skipped_images_count = 0

# Load and send images to function
for idx, (gt_name, pred_name) in enumerate(matched_pairs):
    # Read images with OpenCV
    gt_img = cv2.imread(os.path.join(gt_path, gt_name), 0)  # Load as grayscale
    pred_img = cv2.imread(os.path.join(pred_path, pred_name), 0)  # Load as grayscale
    
    # Check if images are loaded properly
    if gt_img is None or pred_img is None:
        print(f"Error loading images: {gt_name}, {pred_name}")
        continue
    
    gt_img = cv2.resize(gt_img, (224, 224), interpolation=cv2.INTER_NEAREST)  # Use INTER_NEAREST for categorical images
    pred_img = cv2.resize(pred_img, (224, 224), interpolation=cv2.INTER_NEAREST)  # Use INTER_NEAREST for categorical images

    # Convert to torch tensors
    pred_img = torch.from_numpy(pred_img).long()
    gt_img = torch.from_numpy(gt_img).long()

    # Skip if the image contains only two unique classes
    if len(torch.unique(gt_img)) == 2 or len(torch.unique(pred_img)) == 2:
        skipped_images_count += 1
        continue

    # Define mapping: 0 → 0, 76 → 1, 127 → 1, 149 → 2, 255 → 2
    mapping_gt = {0: 0, 76: 1, 127: 1, 149: 2, 255: 2}
    mapping_pred = {0: 0, 127: 1, 255: 2}

    # Apply mapping using np.vectorize
    pred_img = np.vectorize(mapping_pred.get)(pred_img).astype(np.uint8)
    gt_img = np.vectorize(mapping_gt.get)(gt_img).astype(np.uint8)

    # Convert to torch tensors
    pred_img = torch.from_numpy(pred_img).long()
    gt_img = torch.from_numpy(gt_img).long()

    # Angle of Progression Estimation
    gt_result = angle_of_progression_estimation(gt_img, return_img=True)
    pred_result = angle_of_progression_estimation(pred_img, return_img=True)
    
    # Check if either AoP value is None, and skip if true
    if gt_result is None or pred_result is None:
        skipped_images_count += 1
        continue
    
    gtAoP, gtAoPimg = gt_result
    predAoP, predAoPimg = pred_result
    
    # Ensure AoP images are valid (non-empty)
    if gtAoPimg is None or predAoPimg is None or gtAoPimg.size == 0 or predAoPimg.size == 0:
        skipped_images_count += 1
        continue
    
    # Append AoP difference to list
    meanAoP.append(abs(gtAoP - predAoP))
    
    # Append predicted AoP value
    mean_pred_aop.append(predAoP)

# Calculate statistics
if meanAoP:
    mean_diff_aop = np.mean(meanAoP)
    median_diff_aop = np.median(meanAoP)
    std_diff_aop = np.std(meanAoP)
    mean_pred_aop_value = np.mean(mean_pred_aop)
else:
    print("No valid AoP calculations were made.")
    mean_diff_aop = median_diff_aop = std_diff_aop = mean_pred_aop_value = None

# Print the statistics
print(f"Mean Pred AoP: {mean_pred_aop_value:.5f}")
print(f"Mean Diff AoP: {mean_diff_aop:.5f}")
print(f"Median Diff AoP: {median_diff_aop:.5f}")
print(f"Standard Deviation of Diff AoP: {std_diff_aop:.5f}")
print(f"Skipped images count: {skipped_images_count}")


Skipping PS contour due to insufficient points.
Mean Pred AoP: 93.63290
Mean Diff AoP: 4.33484
Median Diff AoP: 1.87167
Standard Deviation of Diff AoP: 5.94611
Skipped images count: 3


## medT

In [10]:
import os
import random

# Paths
gt_path = "/kaggle/input/ground-truth-mask-img/test_Masks-20250329T151233Z-001/test_Masks"
pred_path = "/kaggle/input/final-masks/Pred_Images_for_TN_1414-20250328T162906Z-001/Pred_Images_for_TN_1414/medT_1414/medT/predicted_masks"

# Get filenames
gt_files = sorted(os.listdir(gt_path))
pred_files = sorted(os.listdir(pred_path))

# Remove "pred_" from predicted filenames
clean_pred_files = [fname.replace("pred_", "") for fname in pred_files]

# Find common filenames
common_images = list(set(gt_files) & set(clean_pred_files))

print(f"Total matching masks: {len(common_images)}")  # Check how many match

# Select random matching images
num_images = min(5, len(common_images))  # Load up to 5
random_images = random.sample(common_images, num_images)

# Create matched pairs
matched_pairs = [(img, f"pred_{img}") for img in random_images]

# Print matched pairs for verification
for gt_file, pred_file in matched_pairs:
    print(f"GT: {gt_file}  -->  Pred: {pred_file}")


Total matching masks: 1414
GT: 20200103T102623_713_mask.png  -->  Pred: pred_20200103T102623_713_mask.png
GT: 20191212T102143_59_mask.png  -->  Pred: pred_20191212T102143_59_mask.png
GT: 00177_PSFHAoP4k_mask.png  -->  Pred: pred_00177_PSFHAoP4k_mask.png
GT: 00610_PSFHAoP4k_mask.png  -->  Pred: pred_00610_PSFHAoP4k_mask.png
GT: 01913_PSFHAoP4k_mask.png  -->  Pred: pred_01913_PSFHAoP4k_mask.png


In [11]:
import os
import random
import numpy as np
import cv2  # Import OpenCV
import matplotlib.pyplot as plt
import torch

# Create matched pairs
matched_pairs = [(img, f"pred_{img}") for img in common_images]

meanAoP = []  # For storing the absolute differences
mean_pred_aop = []  # For storing predicted AoP values
skipped_images_count = 0

# Load and send images to function
for idx, (gt_name, pred_name) in enumerate(matched_pairs):
    # Read images with OpenCV
    gt_img = cv2.imread(os.path.join(gt_path, gt_name), 0)  # Load as grayscale
    pred_img = cv2.imread(os.path.join(pred_path, pred_name), 0)  # Load as grayscale
    
    # Check if images are loaded properly
    if gt_img is None or pred_img is None:
        print(f"Error loading images: {gt_name}, {pred_name}")
        continue
    
    gt_img = cv2.resize(gt_img, (224, 224), interpolation=cv2.INTER_NEAREST)  # Use INTER_NEAREST for categorical images
    pred_img = cv2.resize(pred_img, (224, 224), interpolation=cv2.INTER_NEAREST)  # Use INTER_NEAREST for categorical images

    # Convert to torch tensors
    pred_img = torch.from_numpy(pred_img).long()
    gt_img = torch.from_numpy(gt_img).long()

    # Skip if the image contains only two unique classes
    if len(torch.unique(gt_img)) == 2 or len(torch.unique(pred_img)) == 2:
        skipped_images_count += 1
        continue

    # Define mapping: 0 → 0, 76 → 1, 127 → 1, 149 → 2, 255 → 2
    mapping_gt = {0: 0, 76: 1, 127: 1, 149: 2, 255: 2}
    mapping_pred = {0: 0, 127: 1, 255: 2}

    # Apply mapping using np.vectorize
    pred_img = np.vectorize(mapping_pred.get)(pred_img).astype(np.uint8)
    gt_img = np.vectorize(mapping_gt.get)(gt_img).astype(np.uint8)

    # Convert to torch tensors
    pred_img = torch.from_numpy(pred_img).long()
    gt_img = torch.from_numpy(gt_img).long()

    # Angle of Progression Estimation
    gt_result = angle_of_progression_estimation(gt_img, return_img=True)
    pred_result = angle_of_progression_estimation(pred_img, return_img=True)
    
    # Check if either AoP value is None, and skip if true
    if gt_result is None or pred_result is None:
        skipped_images_count += 1
        continue
    
    gtAoP, gtAoPimg = gt_result
    predAoP, predAoPimg = pred_result
    
    # Ensure AoP images are valid (non-empty)
    if gtAoPimg is None or predAoPimg is None or gtAoPimg.size == 0 or predAoPimg.size == 0:
        skipped_images_count += 1
        continue
    
    # Append AoP difference to list
    meanAoP.append(abs(gtAoP - predAoP))
    
    # Append predicted AoP value
    mean_pred_aop.append(predAoP)

# Calculate statistics
if meanAoP:
    mean_diff_aop = np.mean(meanAoP)
    median_diff_aop = np.median(meanAoP)
    std_diff_aop = np.std(meanAoP)
    mean_pred_aop_value = np.mean(mean_pred_aop)
else:
    print("No valid AoP calculations were made.")
    mean_diff_aop = median_diff_aop = std_diff_aop = mean_pred_aop_value = None

# Print the statistics
print(f"Mean Pred AoP: {mean_pred_aop_value:.5f}")
print(f"Mean Diff AoP: {mean_diff_aop:.5f}")
print(f"Median Diff AoP: {median_diff_aop:.5f}")
print(f"Standard Deviation of Diff AoP: {std_diff_aop:.5f}")
print(f"Skipped images count: {skipped_images_count}")


Mean Pred AoP: 96.12322
Mean Diff AoP: 8.37229
Median Diff AoP: 4.73017
Standard Deviation of Diff AoP: 10.43178
Skipped images count: 0


## Raunet

In [12]:
import os
import random

# Paths
gt_path = "/kaggle/input/ground-truth-mask-img/test_Masks-20250329T151233Z-001/test_Masks"
pred_path = "/kaggle/input/final-masks/Pred_Images_for_TN_1414-20250328T162906Z-001/Pred_Images_for_TN_1414/RAUnet_1414/RAUnet/predicted_masks"

# Get filenames
gt_files = sorted(os.listdir(gt_path))
pred_files = sorted(os.listdir(pred_path))

# Remove "pred_" from predicted filenames
clean_pred_files = [fname.replace("pred_", "") for fname in pred_files]

# Find common filenames
common_images = list(set(gt_files) & set(clean_pred_files))

print(f"Total matching masks: {len(common_images)}")  # Check how many match

# Select random matching images
num_images = min(5, len(common_images))  # Load up to 5
random_images = random.sample(common_images, num_images)

# Create matched pairs
matched_pairs = [(img, f"pred_{img}") for img in random_images]

# Print matched pairs for verification
for gt_file, pred_file in matched_pairs:
    print(f"GT: {gt_file}  -->  Pred: {pred_file}")


Total matching masks: 1414
GT: 00817_PSFHAoP4k_mask.png  -->  Pred: pred_00817_PSFHAoP4k_mask.png
GT: 00081_PSFHAoP4k_mask.png  -->  Pred: pred_00081_PSFHAoP4k_mask.png
GT: 01733_PSFHAoP4k_mask.png  -->  Pred: pred_01733_PSFHAoP4k_mask.png
GT: 20191218T105909_449_mask.png  -->  Pred: pred_20191218T105909_449_mask.png
GT: 02963_PSFHAoP4k_mask.png  -->  Pred: pred_02963_PSFHAoP4k_mask.png


In [13]:
import os
import random
import numpy as np
import cv2  # Import OpenCV
import matplotlib.pyplot as plt
import torch

# Create matched pairs
matched_pairs = [(img, f"pred_{img}") for img in common_images]

meanAoP = []  # For storing the absolute differences
mean_pred_aop = []  # For storing predicted AoP values
skipped_images_count = 0

# Load and send images to function
for idx, (gt_name, pred_name) in enumerate(matched_pairs):
    # Read images with OpenCV
    gt_img = cv2.imread(os.path.join(gt_path, gt_name), 0)  # Load as grayscale
    pred_img = cv2.imread(os.path.join(pred_path, pred_name), 0)  # Load as grayscale
    
    # Check if images are loaded properly
    if gt_img is None or pred_img is None:
        print(f"Error loading images: {gt_name}, {pred_name}")
        continue
    
    gt_img = cv2.resize(gt_img, (224, 224), interpolation=cv2.INTER_NEAREST)  # Use INTER_NEAREST for categorical images
    pred_img = cv2.resize(pred_img, (224, 224), interpolation=cv2.INTER_NEAREST)  # Use INTER_NEAREST for categorical images

    # Convert to torch tensors
    pred_img = torch.from_numpy(pred_img).long()
    gt_img = torch.from_numpy(gt_img).long()

    # Skip if the image contains only two unique classes
    if len(torch.unique(gt_img)) == 2 or len(torch.unique(pred_img)) == 2:
        skipped_images_count += 1
        continue

    # Define mapping: 0 → 0, 76 → 1, 127 → 1, 149 → 2, 255 → 2
    mapping_gt = {0: 0, 76: 1, 127: 1, 149: 2, 255: 2}
    mapping_pred = {0: 0, 127: 1, 255: 2}

    # Apply mapping using np.vectorize
    pred_img = np.vectorize(mapping_pred.get)(pred_img).astype(np.uint8)
    gt_img = np.vectorize(mapping_gt.get)(gt_img).astype(np.uint8)

    # Convert to torch tensors
    pred_img = torch.from_numpy(pred_img).long()
    gt_img = torch.from_numpy(gt_img).long()

    # Angle of Progression Estimation
    gt_result = angle_of_progression_estimation(gt_img, return_img=True)
    pred_result = angle_of_progression_estimation(pred_img, return_img=True)
    
    # Check if either AoP value is None, and skip if true
    if gt_result is None or pred_result is None:
        skipped_images_count += 1
        continue
    
    gtAoP, gtAoPimg = gt_result
    predAoP, predAoPimg = pred_result
    
    # Ensure AoP images are valid (non-empty)
    if gtAoPimg is None or predAoPimg is None or gtAoPimg.size == 0 or predAoPimg.size == 0:
        skipped_images_count += 1
        continue
    
    # Append AoP difference to list
    meanAoP.append(abs(gtAoP - predAoP))
    
    # Append predicted AoP value
    mean_pred_aop.append(predAoP)

# Calculate statistics
if meanAoP:
    mean_diff_aop = np.mean(meanAoP)
    median_diff_aop = np.median(meanAoP)
    std_diff_aop = np.std(meanAoP)
    mean_pred_aop_value = np.mean(mean_pred_aop)
else:
    print("No valid AoP calculations were made.")
    mean_diff_aop = median_diff_aop = std_diff_aop = mean_pred_aop_value = None

# Print the statistics
print(f"Mean Pred AoP: {mean_pred_aop_value:.5f}")
print(f"Mean Diff AoP: {mean_diff_aop:.5f}")
print(f"Median Diff AoP: {median_diff_aop:.5f}")
print(f"Standard Deviation of Diff AoP: {std_diff_aop:.5f}")
print(f"Skipped images count: {skipped_images_count}")


Mean Pred AoP: 93.88466
Mean Diff AoP: 3.92404
Median Diff AoP: 2.12935
Standard Deviation of Diff AoP: 5.90912
Skipped images count: 0


## transunet

In [14]:
import os
import random

# Paths
gt_path = "/kaggle/input/ground-truth-mask-img/test_Masks-20250329T151233Z-001/test_Masks"
pred_path = "/kaggle/input/final-masks/Pred_Images_for_TN_1414-20250328T162906Z-001/Pred_Images_for_TN_1414/TransUnet_1414/TransUnet/predicted_masks"

# Get filenames
gt_files = sorted(os.listdir(gt_path))
pred_files = sorted(os.listdir(pred_path))

# Remove "pred_" from predicted filenames
clean_pred_files = [fname.replace("pred_", "") for fname in pred_files]

# Find common filenames
common_images = list(set(gt_files) & set(clean_pred_files))

print(f"Total matching masks: {len(common_images)}")  # Check how many match

# Select random matching images
num_images = min(5, len(common_images))  # Load up to 5
random_images = random.sample(common_images, num_images)

# Create matched pairs
matched_pairs = [(img, f"pred_{img}") for img in random_images]

# Print matched pairs for verification
for gt_file, pred_file in matched_pairs:
    print(f"GT: {gt_file}  -->  Pred: {pred_file}")


Total matching masks: 1414
GT: 00635_PSFHAoP4k_mask.png  -->  Pred: pred_00635_PSFHAoP4k_mask.png
GT: 00190_PSFHAoP4k_mask.png  -->  Pred: pred_00190_PSFHAoP4k_mask.png
GT: 03959_PSFHAoP1358_mask.png  -->  Pred: pred_03959_PSFHAoP1358_mask.png
GT: 03621_PSFHAoP4k_mask.png  -->  Pred: pred_03621_PSFHAoP4k_mask.png
GT: 20190918T123342_949_mask.png  -->  Pred: pred_20190918T123342_949_mask.png


In [15]:
import os
import random
import numpy as np
import cv2  # Import OpenCV
import matplotlib.pyplot as plt
import torch

# Create matched pairs
matched_pairs = [(img, f"pred_{img}") for img in common_images]

meanAoP = []  # For storing the absolute differences
mean_pred_aop = []  # For storing predicted AoP values
skipped_images_count = 0

# Load and send images to function
for idx, (gt_name, pred_name) in enumerate(matched_pairs):
    # Read images with OpenCV
    gt_img = cv2.imread(os.path.join(gt_path, gt_name), 0)  # Load as grayscale
    pred_img = cv2.imread(os.path.join(pred_path, pred_name), 0)  # Load as grayscale
    
    # Check if images are loaded properly
    if gt_img is None or pred_img is None:
        print(f"Error loading images: {gt_name}, {pred_name}")
        continue
    
    gt_img = cv2.resize(gt_img, (224, 224), interpolation=cv2.INTER_NEAREST)  # Use INTER_NEAREST for categorical images
    pred_img = cv2.resize(pred_img, (224, 224), interpolation=cv2.INTER_NEAREST)  # Use INTER_NEAREST for categorical images

    # Convert to torch tensors
    pred_img = torch.from_numpy(pred_img).long()
    gt_img = torch.from_numpy(gt_img).long()

    # Skip if the image contains only two unique classes
    if len(torch.unique(gt_img)) == 2 or len(torch.unique(pred_img)) == 2:
        skipped_images_count += 1
        continue

    # Define mapping: 0 → 0, 76 → 1, 127 → 1, 149 → 2, 255 → 2
    mapping_gt = {0: 0, 76: 1, 127: 1, 149: 2, 255: 2}
    mapping_pred = {0: 0, 127: 1, 255: 2}

    # Apply mapping using np.vectorize
    pred_img = np.vectorize(mapping_pred.get)(pred_img).astype(np.uint8)
    gt_img = np.vectorize(mapping_gt.get)(gt_img).astype(np.uint8)

    # Convert to torch tensors
    pred_img = torch.from_numpy(pred_img).long()
    gt_img = torch.from_numpy(gt_img).long()

    # Angle of Progression Estimation
    gt_result = angle_of_progression_estimation(gt_img, return_img=True)
    pred_result = angle_of_progression_estimation(pred_img, return_img=True)
    
    # Check if either AoP value is None, and skip if true
    if gt_result is None or pred_result is None:
        skipped_images_count += 1
        continue
    
    gtAoP, gtAoPimg = gt_result
    predAoP, predAoPimg = pred_result
    
    # Ensure AoP images are valid (non-empty)
    if gtAoPimg is None or predAoPimg is None or gtAoPimg.size == 0 or predAoPimg.size == 0:
        skipped_images_count += 1
        continue
    
    # Append AoP difference to list
    meanAoP.append(abs(gtAoP - predAoP))
    
    # Append predicted AoP value
    mean_pred_aop.append(predAoP)

# Calculate statistics
if meanAoP:
    mean_diff_aop = np.mean(meanAoP)
    median_diff_aop = np.median(meanAoP)
    std_diff_aop = np.std(meanAoP)
    mean_pred_aop_value = np.mean(mean_pred_aop)
else:
    print("No valid AoP calculations were made.")
    mean_diff_aop = median_diff_aop = std_diff_aop = mean_pred_aop_value = None

# Print the statistics
print(f"Mean Pred AoP: {mean_pred_aop_value:.5f}")
print(f"Mean Diff AoP: {mean_diff_aop:.5f}")
print(f"Median Diff AoP: {median_diff_aop:.5f}")
print(f"Standard Deviation of Diff AoP: {std_diff_aop:.5f}")
print(f"Skipped images count: {skipped_images_count}")


Mean Pred AoP: 94.30335
Mean Diff AoP: 3.83576
Median Diff AoP: 2.28318
Standard Deviation of Diff AoP: 4.26828
Skipped images count: 0


## transfuse

In [16]:
import os
import random

# Paths
gt_path = "/kaggle/input/ground-truth-mask-img/test_Masks-20250329T151233Z-001/test_Masks"
pred_path = "/kaggle/input/final-masks/Pred_Images_for_TN_1414-20250328T162906Z-001/Pred_Images_for_TN_1414/TransFuse_1414/TransFuse/predicted_masks"

# Get filenames
gt_files = sorted(os.listdir(gt_path))
pred_files = sorted(os.listdir(pred_path))

# Remove "pred_" from predicted filenames
clean_pred_files = [fname.replace("pred_", "") for fname in pred_files]

# Find common filenames
common_images = list(set(gt_files) & set(clean_pred_files))

print(f"Total matching masks: {len(common_images)}")  # Check how many match

# Select random matching images
num_images = min(5, len(common_images))  # Load up to 5
random_images = random.sample(common_images, num_images)

# Create matched pairs
matched_pairs = [(img, f"pred_{img}") for img in random_images]

# Print matched pairs for verification
for gt_file, pred_file in matched_pairs:
    print(f"GT: {gt_file}  -->  Pred: {pred_file}")


Total matching masks: 1414
GT: 02654_PSFHAoP4k_mask.png  -->  Pred: pred_02654_PSFHAoP4k_mask.png
GT: 20190909T161453_219_mask.png  -->  Pred: pred_20190909T161453_219_mask.png
GT: 03988_PSFHAoP1358_mask.png  -->  Pred: pred_03988_PSFHAoP1358_mask.png
GT: 00675_PSFHAoP4k_mask.png  -->  Pred: pred_00675_PSFHAoP4k_mask.png
GT: 02161_PSFHAoP4k_mask.png  -->  Pred: pred_02161_PSFHAoP4k_mask.png


In [17]:
import os
import random
import numpy as np
import cv2  # Import OpenCV
import matplotlib.pyplot as plt
import torch

# Create matched pairs
matched_pairs = [(img, f"pred_{img}") for img in common_images]

meanAoP = []  # For storing the absolute differences
mean_pred_aop = []  # For storing predicted AoP values
skipped_images_count = 0

# Load and send images to function
for idx, (gt_name, pred_name) in enumerate(matched_pairs):
    # Read images with OpenCV
    gt_img = cv2.imread(os.path.join(gt_path, gt_name), 0)  # Load as grayscale
    pred_img = cv2.imread(os.path.join(pred_path, pred_name), 0)  # Load as grayscale
    
    # Check if images are loaded properly
    if gt_img is None or pred_img is None:
        print(f"Error loading images: {gt_name}, {pred_name}")
        continue
    
    gt_img = cv2.resize(gt_img, (224, 224), interpolation=cv2.INTER_NEAREST)  # Use INTER_NEAREST for categorical images
    pred_img = cv2.resize(pred_img, (224, 224), interpolation=cv2.INTER_NEAREST)  # Use INTER_NEAREST for categorical images

    # Convert to torch tensors
    pred_img = torch.from_numpy(pred_img).long()
    gt_img = torch.from_numpy(gt_img).long()

    # Skip if the image contains only two unique classes
    if len(torch.unique(gt_img)) == 2 or len(torch.unique(pred_img)) == 2:
        skipped_images_count += 1
        continue

    # Define mapping: 0 → 0, 76 → 1, 127 → 1, 149 → 2, 255 → 2
    mapping_gt = {0: 0, 76: 1, 127: 1, 149: 2, 255: 2}
    mapping_pred = {0: 0, 127: 1, 255: 2}

    # Apply mapping using np.vectorize
    pred_img = np.vectorize(mapping_pred.get)(pred_img).astype(np.uint8)
    gt_img = np.vectorize(mapping_gt.get)(gt_img).astype(np.uint8)

    # Convert to torch tensors
    pred_img = torch.from_numpy(pred_img).long()
    gt_img = torch.from_numpy(gt_img).long()

    # Angle of Progression Estimation
    gt_result = angle_of_progression_estimation(gt_img, return_img=True)
    pred_result = angle_of_progression_estimation(pred_img, return_img=True)
    
    # Check if either AoP value is None, and skip if true
    if gt_result is None or pred_result is None:
        skipped_images_count += 1
        continue
    
    gtAoP, gtAoPimg = gt_result
    predAoP, predAoPimg = pred_result
    
    # Ensure AoP images are valid (non-empty)
    if gtAoPimg is None or predAoPimg is None or gtAoPimg.size == 0 or predAoPimg.size == 0:
        skipped_images_count += 1
        continue
    
    # Append AoP difference to list
    meanAoP.append(abs(gtAoP - predAoP))
    
    # Append predicted AoP value
    mean_pred_aop.append(predAoP)

# Calculate statistics
if meanAoP:
    mean_diff_aop = np.mean(meanAoP)
    median_diff_aop = np.median(meanAoP)
    std_diff_aop = np.std(meanAoP)
    mean_pred_aop_value = np.mean(mean_pred_aop)
else:
    print("No valid AoP calculations were made.")
    mean_diff_aop = median_diff_aop = std_diff_aop = mean_pred_aop_value = None

# Print the statistics
print(f"Mean Pred AoP: {mean_pred_aop_value:.5f}")
print(f"Mean Diff AoP: {mean_diff_aop:.5f}")
print(f"Median Diff AoP: {median_diff_aop:.5f}")
print(f"Standard Deviation of Diff AoP: {std_diff_aop:.5f}")
print(f"Skipped images count: {skipped_images_count}")

Skipping PS contour due to insufficient points.
Mean Pred AoP: 95.20934
Mean Diff AoP: 8.51317
Median Diff AoP: 2.91189
Standard Deviation of Diff AoP: 12.78325
Skipped images count: 5


## attentionUnet

In [18]:
import os
import random

# Paths
gt_path = "/kaggle/input/ground-truth-mask-img/test_Masks-20250329T151233Z-001/test_Masks"
pred_path = "/kaggle/input/final-masks/Pred_Images_for_TN_1414-20250328T162906Z-001/Pred_Images_for_TN_1414/AttentionUnet_1414/AttentionUnet/predicted_masks"

# Get filenames
gt_files = sorted(os.listdir(gt_path))
pred_files = sorted(os.listdir(pred_path))

# Remove "pred_" from predicted filenames
clean_pred_files = [fname.replace("pred_", "") for fname in pred_files]

# Find common filenames
common_images = list(set(gt_files) & set(clean_pred_files))

print(f"Total matching masks: {len(common_images)}")  # Check how many match

# Select random matching images
num_images = min(5, len(common_images))  # Load up to 5
random_images = random.sample(common_images, num_images)

# Create matched pairs
matched_pairs = [(img, f"pred_{img}") for img in random_images]

# Print matched pairs for verification
for gt_file, pred_file in matched_pairs:
    print(f"GT: {gt_file}  -->  Pred: {pred_file}")


Total matching masks: 1414
GT: 20191214T103803_649_mask.png  -->  Pred: pred_20191214T103803_649_mask.png
GT: 20191008T112326_109_mask.png  -->  Pred: pred_20191008T112326_109_mask.png
GT: 20200103T102728_429_mask.png  -->  Pred: pred_20200103T102728_429_mask.png
GT: 02230_PSFHAoP4k_mask.png  -->  Pred: pred_02230_PSFHAoP4k_mask.png
GT: 00053_PSFHAoP4k_mask.png  -->  Pred: pred_00053_PSFHAoP4k_mask.png


In [19]:
import os
import random
import numpy as np
import cv2  # Import OpenCV
import matplotlib.pyplot as plt
import torch

# Create matched pairs
matched_pairs = [(img, f"pred_{img}") for img in common_images]

meanAoP = []  # For storing the absolute differences
mean_pred_aop = []  # For storing predicted AoP values
skipped_images_count = 0

# Load and send images to function
for idx, (gt_name, pred_name) in enumerate(matched_pairs):
    # Read images with OpenCV
    gt_img = cv2.imread(os.path.join(gt_path, gt_name), 0)  # Load as grayscale
    pred_img = cv2.imread(os.path.join(pred_path, pred_name), 0)  # Load as grayscale
    
    # Check if images are loaded properly
    if gt_img is None or pred_img is None:
        print(f"Error loading images: {gt_name}, {pred_name}")
        continue
    
    gt_img = cv2.resize(gt_img, (224, 224), interpolation=cv2.INTER_NEAREST)  # Use INTER_NEAREST for categorical images
    pred_img = cv2.resize(pred_img, (224, 224), interpolation=cv2.INTER_NEAREST)  # Use INTER_NEAREST for categorical images

    # Convert to torch tensors
    pred_img = torch.from_numpy(pred_img).long()
    gt_img = torch.from_numpy(gt_img).long()

    # Skip if the image contains only two unique classes
    if len(torch.unique(gt_img)) == 2 or len(torch.unique(pred_img)) == 2:
        skipped_images_count += 1
        continue

    # Define mapping: 0 → 0, 76 → 1, 127 → 1, 149 → 2, 255 → 2
    mapping_gt = {0: 0, 76: 1, 127: 1, 149: 2, 255: 2}
    mapping_pred = {0: 0, 127: 1, 255: 2}

    # Apply mapping using np.vectorize
    pred_img = np.vectorize(mapping_pred.get)(pred_img).astype(np.uint8)
    gt_img = np.vectorize(mapping_gt.get)(gt_img).astype(np.uint8)

    # Convert to torch tensors
    pred_img = torch.from_numpy(pred_img).long()
    gt_img = torch.from_numpy(gt_img).long()

    # Angle of Progression Estimation
    gt_result = angle_of_progression_estimation(gt_img, return_img=True)
    pred_result = angle_of_progression_estimation(pred_img, return_img=True)
    
    # Check if either AoP value is None, and skip if true
    if gt_result is None or pred_result is None:
        skipped_images_count += 1
        continue
    
    gtAoP, gtAoPimg = gt_result
    predAoP, predAoPimg = pred_result
    
    # Ensure AoP images are valid (non-empty)
    if gtAoPimg is None or predAoPimg is None or gtAoPimg.size == 0 or predAoPimg.size == 0:
        skipped_images_count += 1
        continue
    
    # Append AoP difference to list
    meanAoP.append(abs(gtAoP - predAoP))
    
    # Append predicted AoP value
    mean_pred_aop.append(predAoP)

# Calculate statistics
if meanAoP:
    mean_diff_aop = np.mean(meanAoP)
    median_diff_aop = np.median(meanAoP)
    std_diff_aop = np.std(meanAoP)
    mean_pred_aop_value = np.mean(mean_pred_aop)
else:
    print("No valid AoP calculations were made.")
    mean_diff_aop = median_diff_aop = std_diff_aop = mean_pred_aop_value = None

# Print the statistics
print(f"Mean Pred AoP: {mean_pred_aop_value:.5f}")
print(f"Mean Diff AoP: {mean_diff_aop:.5f}")
print(f"Median Diff AoP: {median_diff_aop:.5f}")
print(f"Standard Deviation of Diff AoP: {std_diff_aop:.5f}")
print(f"Skipped images count: {skipped_images_count}")


Mean Pred AoP: 94.75545
Mean Diff AoP: 3.83096
Median Diff AoP: 1.82738
Standard Deviation of Diff AoP: 4.99525
Skipped images count: 0
